In [20]:
import pandas as pd
import pyarrow.feather as feather
from sklearn import pipeline, preprocessing, feature_selection
import numpy as np
import datetime
%matplotlib inline

# Lectura de archivos

In [21]:
raw_paint_square_meters_df = pd.read_excel(
    '../ternium-data/Exp_2_Pintura_Estructura Analisis de consumo y rendimiento mensual.xlsx',
    sheet_name='Metros')
raw_paint_performance_df = pd.read_excel(
    '../ternium-data/Exp_2_Pintura_Estructura Analisis de consumo y rendimiento mensual.xlsx',
    sheet_name='Rendimientos INDU')

raw_coating_2021_dfs = [
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo agosto 1 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo agosto 2 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo septiembre 1 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo septiembre 2 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo octubre 1 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo octubre 2 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo noviembre 1 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo noviembre 2 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo diciembre 1 uni 2021.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Revestidos/Consumo diciembre 2 uni 2021.xlsx',
    ),
]
raw_coating_20_23_df = pd.read_excel('../ternium-data/P12. Pinturas Revestidos Mx Jul20-Ago23_psch.xlsx',
                                     sheet_name='Pinturas Revestidos Mx Ene21 a ', header=1)

raw_production_p1_dfs = [
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Enero_Diciembre_2021_Pintado1_UNI.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Enero_Diciembre_2022_Pintado1_UNI.xlsx'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_MarzoPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_AbrilPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_MayoPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_JunioPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_JulioPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_AgostoPintado1.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Sep2023_Abr2024_UNI_1.xlsx'
    ),
]

raw_production_p2_dfs = [
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Enero_Diciembre_2021_Pintado2_UNI.xlsx',
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Enero_Diciembre_2022_Pintado2_UNI.xlsx'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_MarzoPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_AbrilPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_MayoPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_JunioPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_JulioPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_AgostoPintado2.xlsx',
        sheet_name='ResumenProduccion'
    ),
    pd.read_excel(
        '../ternium-data/Producción/ResumenProduccion_Sep2023_Abr2024_UNI_2.xlsx'
    ),
]

raw_paint_catalog_df = pd.read_excel('../ternium-data/Catálogo de colores 02_10_23.xlsx')

# Utility functions

In [22]:
def convert_object_cols_to_category(df):
    for name in df.columns:
        if df[name].dtype == np.dtype('O'):
            display(f'converted {name} to categorical column')
            df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


def print_numeric_columns(df):
    for name in df._get_numeric_data().columns:
        display(name)


def get_numeric_columns(df):
    columns_names = []
    for name in df._get_numeric_data().columns:
        columns_names.append(name)
    return columns_names


class CutoffTransformer:
    cutoff = 0.0

    def __init__(self, cutoff):
        self.cutoff = cutoff

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.sign(X) * (np.minimum(np.abs(X), [self.cutoff]))

    def get_feature_names_out(self, input_features=None):
        return input_features


class NullColumnRemover:
    null_threshold = 0.0
    columns_to_keep = None

    def __init__(self, null_threshold):
        self.null_threshold = null_threshold

    def fit(self, X, y=None):
        null_percents = pd.isna(X).sum(axis=0) / X.shape[0]
        self.columns_to_keep = null_percents[null_percents < self.null_threshold].index
        return self

    def transform(self, X):
        return X[self.columns_to_keep]

    def get_feature_names_out(self, input_features=None):
        return self.columns_to_keep.values

# Defect processing setup

In [23]:
defects_cols = [
    'Codigo defecto',
    'Defecto',
    'Ubicacion',
    'Es Contencion',
    'Es Prevencion',
    'Es Mejora',
    'Intensidad',
    'Cara',
    'Lado',
    'Linea Origen',
    'Dictamen',
    'Resolución',
    'Calidad Sugerida',
    'Siguiente Accion',
    'Siguiente Proceso',
    'Mts Ini',
    'Mts Fin',
    'Frecuencia',
    'Causa',
    'Accion Preventiva',
    'Accion Correctiva',
    'Observaciones',
    'Fecha Registro',
    'Registro Automatico',
    'Código Equipo',
]


def extract_defects_df(production_df: pd.DataFrame):
    """
    This function aims to extract defects data from a production DataFrame, assuming it has specified columns labeled
    with defect types and corresponding numerical identifiers in brackets like ' (1)', ' (2)', etc. for multiple observations.

    It drops NA values and concatenates the data to form a unified DataFrame that captures all the defects detail.
    The final DataFrame is indexed serially.

    Parameters:
    
    production_df (DataFrame): pandas DataFrame with production data, expected to include defects_cols with numerical identifiers.

    Returns:
    defects_df: A DataFrame containing the combined defects data from all the identities.
    """
    defects_df_1 = production_df[map(lambda x: x + ' (1)', defects_cols)].dropna(thresh=1)
    defects_df_1.columns = defects_cols
    defects_df_2 = production_df[map(lambda x: x + ' (2)', defects_cols)].dropna(thresh=1)
    defects_df_2.columns = defects_cols
    defects_df_3 = production_df[map(lambda x: x + ' (3)', defects_cols)].dropna(thresh=1)
    defects_df_3.columns = defects_cols
    defects_df_4 = production_df[map(lambda x: x + ' (4)', defects_cols)].dropna(thresh=1)
    defects_df_4.columns = defects_cols
    defects_df_5 = production_df[map(lambda x: x + ' (5)', defects_cols)].dropna(thresh=1)
    defects_df_5.columns = defects_cols

    # the defect ID is the same as the id for its respective production line. As such, we can extract
    # it using reset_index to reference the original production line id in a new column.
    # then, the resulting dfs are concatenated while ignoring the existing index.
    defects_df = pd.concat([
        defects_df_1.reset_index(names='production_id'),
        defects_df_2.reset_index(names='production_id'),
        defects_df_3.reset_index(names='production_id'),
        defects_df_4.reset_index(names='production_id'),
        defects_df_5.reset_index(names='production_id'),
    ], ignore_index=True)

    return defects_df


def remove_defects_cols(production_id: pd.DataFrame):
    """
    Remove defects columns from the given production_id DataFrame.

    :param production_id: The DataFrame containing the production data.
    :return: The DataFrame with defects columns removed.
    """
    columns_to_remove = list(map(lambda x: x + ' (1)', defects_cols)) + list(
        map(lambda x: x + ' (2)', defects_cols)) + list(map(lambda x: x + ' (3)', defects_cols)) + list(
        map(lambda x: x + ' (4)', defects_cols)) + list(map(lambda x: x + ' (5)', defects_cols))
    return production_id.drop(columns=columns_to_remove)


# Join coating tables 

In [24]:
raw_coating_df = pd.concat([raw_coating_20_23_df] + raw_coating_2021_dfs, ignore_index=True)

# Join production tables and extract defects

In [25]:
raw_production_p1_df = pd.concat(raw_production_p1_dfs, ignore_index=True).assign(production_line='Pintado 1')
raw_production_p2_df = pd.concat(raw_production_p2_dfs, ignore_index=True).assign(production_line='Pintado 2')

raw_production_df = pd.concat([raw_production_p1_df, raw_production_p2_df], ignore_index=True)

# extract defects from production summary
raw_defects_df = extract_defects_df(raw_production_df)

raw_production_df = remove_defects_cols(raw_production_df)

/tmp/ipykernel_95769/2335043041.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  raw_production_p1_df = pd.concat(raw_production_p1_dfs, ignore_index=True).assign(production_line='Pintado 1')
/tmp/ipykernel_95769/2335043041.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  raw_production_p1_df = pd.concat(raw_production_p1_dfs, ignore_index=True).assign(production_line='Pintado 1')
/tmp/ipykernel_95769/2335043041.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all

# Data preprocessing
## For paint performance dataframe

In [26]:
paint_performance_df = raw_paint_performance_df.rename(columns={
    'Pintura': 'paint_name',
    'Rendimiento Canning [m2/L]': 'expected_yield'
}, errors='raise'
)[['paint_name', 'expected_yield']].groupby('paint_name').mean()

## For production dataframe

## For defects dataframe

In [27]:
# Rename all columns from the dataframe
production_df = raw_production_df.rename(columns={
    'Material Entrada': 'input_material_code',
    'Material Salida': 'output_material_code',
    'Fecha Inicio': 'start_date',
    'Fecha Fin': 'end_date',
    'Turno': 'shift',
    'Estado': 'state',
    'Necesidad': 'need',
    'Cliente': 'client_name',
    'Clase': 'paint_class',
    'Grado': 'grade',
    'Norma': 'norm',
    'Uso Final': 'final_usage',
    'Uso General': 'general_usage',
    'Esquema Pintado': 'paint_schema',
    'Esquema Pintado RF': 'paint_schema_rf',
    'Empaque': 'packing',
    'Código Clear Inf': 'inferior_clear_code',
    'Código Clear Sup': 'superior_clear_code',
    'Ancho 1': 'width1_mm',
    'Ancho 2': 'width2_mm',
    'Ancho 3': 'width3_mm',
    'Ancho': 'width_mm',
    'Espesor 1': 'thickness1_mm',
    'Espesor 2': 'thickness2_mm',
    'Espesor 3': 'thickness3_mm',
    'Espesor': 'thickness_mm',
    'Peso Entrada': 'input_weight_kg',
    'Peso': 'weight_kg',
    'Balance Forzado': 'forced_balance',
    'Usuario Balance Peso': 'forced_balance_by_user',
    'Usuario Registro Peso': 'registered_weight_by_user',
    'Largo': 'length_m',
    'Largo PLC': 'length_plc',
    'Línea Siguiente': 'next_line',
    'PRAM': 'pram',
    'Color Inferior': 'inferior_color_code',
    'Color Superior': 'superior_color_code',
    'Primer Superior': 'superior_primer_code',
    'Primer Inferior': 'inferior_primer_code',
    'Peso Mínimo': 'min_weight',
    'Peso Máximo': 'max_weight',
    'Calibre': 'caliber',
    'Código Primer Inf': 'primer_code_inf',
    'Código Primer Sup': 'primer_code_sup',
    'Capa Acabado Inf': 'finish_coat_inf',
    'Capa Acabado LO Inf': 'finish_coat_lo_inf',
    'Capa Acabado LM Inf': 'finish_coat_lm_inf',
    'Capa Acabado Sup': 'finish_coat_sup',
    'Capa Acabado LO Sup': 'finish_coat_lo_sup',
    'Capa Acabado LM Sup': 'finish_coat_lm_sup',
    'Capa Primer Inf': 'primer_coat_inf',
    'Capa Primer LO Inf': 'primer_coat_lo_inf',
    'Capa Primer LM Inf': 'primer_coat_lm_inf',
    'Capa Primer Sup': 'primer_coat_sup',
    'Capa Primer LO Sup': 'primer_coat_lo_sup',
    'Capa Primer LM Sup': 'primer_coat_lm_sup',
    'Capa Clear Inf': 'clear_coat_inf',
    'Capa Clear LO Inf': 'clear_coat_lo_inf',
    'Capa Clear MN Inf': 'clear_coat_mn_inf',
    'Capa Clear Sup': 'clear_coat_sup',
    'CapaClear LO Sup': 'clear_coat_lo_sup',
    'Capa Clear LM Sup': 'clear_coat_lm_inf',
    'Brillo Sup': 'gloss_sup',
    'Brillo Inf': 'gloss_inf',
    'Grado Brillo Sup': 'gloss_grade_sup',
    'Grado Brillo Inf': 'gloss_grade_inf',
    'Dureza Pintura LO Sup': 'paint_toughness_lo_sup',
    'Dureza Pintura LM Sup': 'paint_toughness_lm_sup',
    'Dureza Pintura Sup': 'paint_toughness_sup',
    'Dureza Pintura LO Inf': 'paint_toughness_lo_inf',
    'Dureza Pintura LM Inf': 'paint_toughness_lm_inf',
    'Dureza Pintura Inf': 'paint_toughness_inf',
    'Flexión Pintura NMC Inf': 'paint_flex_nmc_inf',
    'Flexión Pintura NMC Sup': 'paint_flex_nmc_sup',
    'Flexión Pintura LO Inf': 'paint_flex_lo_inf',
    'Flexión Pintura LM Inf': 'paint_flex_lm_inf',
    'Flexión Pintura Inf': 'paint_flex_inf',
    'Flexión Pintura LO Sup': 'paint_flex_lo_sup',
    'Flexión Pintura LM Sup': 'paint_flex_lm_sup',
    'Flexión Pintura Sup': 'paint_flex_up',
    'Impacto Pintura Inf': 'paint_impact_inf',
    'Impacto Pintura LO Inf': 'paint_impact_lo_inf',
    'Impacto Pintura LM Inf': 'paint_impact_lm_inf',
    'Impacto Pintura Sup': 'paint_impact_sup',
    'Impacto Pintura LO Sup': 'paint_impact_lo_sup',
    'Impacto Pintura LM Sup': 'paint_impact_lm_sup',
    'Resistencia Solvente Sup': 'solvent_resistance_sup',
    'Resistencia Solvente Inf': 'solvent_resistance_inf',
    'Análisis Tono DA Inf': 'tone_analysis_da_inf',
    'Análisis Tono DA Sup': 'tone_analysis_da_sup',
    'Análisis Tono DB Inf': 'tone_analysis_db_inf',
    'Análisis Tono DB Sup': 'tone_analysis_db_sup',
    'Análisis Tono DC Inf': 'tone_analysis_dc_inf',
    'Análisis Tono DC Sup': 'tone_analysis_dc_sup',
    'Análisis Tono DE Inf': 'tone_analysis_de_inf',
    'Análisis Tono DE Sup': 'tone_analysis_de_sup',
    'Análisis Tono DL Inf': 'tone_analysis_dl_inf',
    'Análisis Tono DL Sup': 'tone_analysis_dl_sup',
    'Polietileno Código': 'polyethylene_code',
    'Polietileno Metros Aplicados': 'polyethylene_applied_meters',
    'Polietileno Adherencia': 'polyethylene_adherence',
    'Polietileno Ancho': 'polyethylene_width',
    'Polietileno Tipo Aplicacion': 'polyethylene_application_type',
    'Velocidad Promedio': 'average_speed',
    'Cinta de Noche': 'night_shift',
    'Mercado Objetivo': 'target_market',
    '% Línea Operando': 'operating_line_percent',
    '% Primer Sup. Salida': 'primer_sup_exit_percent',
    '% Primer Inf. Salida': 'primer_inf_exit_percent',
    '% Cromato Sup Salida': 'chromate_sup_exit_percent',
    '% Cromato Inf Salida': 'chromate_inf_exit_percent',
    '% Acabado 1 Sup Salida': 'finish_sup_exit_percent',
    '% Acabado 1 Inf Salida': 'finish_inf_exit_percent',
    '% Uso Cabezales': 'heads_usage_percent',
    'Hizo Proceso NO STD': 'process_no_std',
    'Programa': 'program',
    'Vel. Proceso Promedio': 'average_process_speed',
    'Resolución': 'resolution',
    'Dictamen': 'final_decision',
    'Calidad Sugerida': 'suggested_quality',
    'ID Red': 'network_id',
    'Usuario': 'user',
    'Es Reproceso': 'is_reprocess',
    'Muestrear': 'sample',
    'Muestreado': 'sampled',
    'Ruta Teórica': 'route',
    '% Acabado Inf Salida': 'finish_2_inf_exit_percent',
    '% Acabado Sup Norte Salida': 'finish_sup_north_exit_percent',
    '% Acabado Sup Sur Salida': 'finish_sup_south_exit_percent'
}, errors='raise')

production_df = production_df.drop(['input_material_code', 'output_material_code'], axis='columns')

production_df = production_df.assign(
    shift=lambda df: df['shift'].round().astype('category'),
    is_reprocess=lambda df: df['is_reprocess'].round().astype('category'),
    primer_code_inf=lambda df: df['primer_code_inf'].astype('category'),
    primer_code_sup=lambda df: df['primer_code_sup'].astype('category'),
    program=lambda df: df['program'].astype('category'),
    weight_ratio=lambda df: df['weight_kg'] / df['input_weight_kg'],
    m2=lambda df: df['length_m'] * (df['width_mm'] / 1000),
    need=lambda df: (df['need'] / 1000000000).round(1).astype('category'),
)

convert_object_cols_to_category(production_df)

'converted state to categorical column'

'converted client_name to categorical column'

'converted paint_class to categorical column'

'converted grade to categorical column'

'converted norm to categorical column'

...

'converted sampled to categorical column'

'converted route to categorical column'

'converted Hizo Proceso NO STD      to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted production_line to categorical column'

## For coating dataframe

In [28]:
defects_df = raw_defects_df.rename(columns={
    'Codigo defecto': 'defect_code',
    'Defecto': 'defect_name',
    'Ubicacion': 'location',
    'Es Contencion': 'is_containment',
    'Es Prevencion': 'is_preventive',
    'Es Mejora': 'is_improvement',
    'Intensidad': 'intensity',
    'Cara': 'face',
    'Lado': 'side',
    'Linea Origen': 'origin_line',
    'Dictamen': 'final_decison',
    'Resolución': 'resolution',
    'Calidad Sugerida': 'suggested_quality',
    'Siguiente Accion': 'next_action',
    'Siguiente Proceso': 'next_process',
    'Mts Ini': 'initial_MTS',
    'Mts Fin': 'final_MTS',
    'Frecuencia': 'frequency',
    'Fecha Registro': 'register_date',
    'Registro Automatico': 'automatic_registration',
}, errors='raise')

defects_df = defects_df.assign(
    production_id=lambda df: df['production_id'].astype('category'),
    defect_code=lambda df: df['defect_code'].astype('category'),
    is_containment=lambda df: df['is_containment'].astype('category'),
    is_preventive=lambda df: df['is_preventive'].astype('category'),
    is_improvement=lambda df: df['is_improvement'].astype('category'),
    intensity=lambda df: df['intensity'].astype('category'),
    next_action=lambda df: df['next_action'].astype('category'),
    next_process=lambda df: df['next_process'].astype('category'),
    automatic_registration=lambda df: df['production_id'].astype('category'),
)
convert_object_cols_to_category(defects_df)

'converted defect_name to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted location to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted face to categorical column'

...

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted Observaciones to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted Código Equipo to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


In [29]:
coating_df = raw_coating_df.rename(columns={
    'Soc.': 'soc',
    'Ce.coste': 'ce_cost',
    'Cl.coste': 'cost_ci',
    'DenomClCst': 'denom_cl_cst',
    'Cta.CP': 'cta_cp',
    'Denominación de la cuenta de contraparti': 'account_denomination',
    'Denominación objeto': 'production_line',  #
    'Material': 'paint_id',  #
    'Texto breve de material': 'paint_name',  #
    'OrdPartner': 'order_partner',
    'Doc.compr.': 'document_sale',
    'Pos.': 'pos',
    'Txt.pedido': 'order_text',
    'Denominación': 'denomination',
    'Cl.doc': 'cl_doc ',
    'Nº doc.': 'document_number',
    'Nºdoc.ref.': 'document_number_reference',
    'Acreedor': 'creditor',
    'Nom.1': 'nom_1',
    'Usuario': 'user',
    'Per': 'period',
    ' Año': 'year',
    '  Valor/Mon.tr.': 'transaction_monetary_value',
    'MonT': 'currency_transaction',
    ' Valor/Mon.obj.': 'object_monetary_value',
    'MonO': 'currency_object',
    '    Valor var.': 'monetary_value_usd',  #
    'MSoCO': 'currency_variation',
    'Ctd.total reg.': 'total_liters_used',  # 
    'UCc': 'cost_center',
    'Fecha doc.': 'document_date',
    'Fe.contab.': 'accounting_date',
    'Hora': 'time',
    'Registrado': 'registered',
    'Asignación': 'assignment',
    'Planta': 'production_plant',  #
    'Mes': 'month',
    'Año': 'year_duplicated',
    'Proveedor': 'supplier',  #
    'Registrado': 'date',  #
    'Cliente': 'client',
    'Precio': 'price_per_liter',  #
    'Línea': 'line',  #
}, errors='raise')

coating_df = coating_df.drop(
    ['month', 'year', 'year_duplicated', 'document_number_reference', 'order_text', 'document_number'], axis='columns')

coating_df = coating_df[coating_df['production_plant'] == 'Uni']

coating_df = coating_df.assign(
    date=lambda df: pd.to_datetime(df['date'], errors='coerce'),

    production_line=lambda df: df['production_line'].replace({
        'Pintado 1 Prod. Univ': 'Pintado 1',
        'Pintado 2 Prod. Univ': 'Pintado 2',
        'Pintado 1 UNI': 'Pintado 1',
        'Pintado 2 UNI': 'Pintado 2',
    }).astype('category'),
    ce_cost=lambda df: df['ce_cost'].astype('category'),
    period=lambda df: df['period'].astype('category'),
    order_partner=lambda df: df['order_partner'].astype('category'),
    document_sale=lambda df: df['document_sale'].astype('category'),
    creditor=lambda df: df['creditor'].astype('category'),
    time=lambda df: df['time'].apply(
        lambda value: datetime.datetime.combine(datetime.date.min, value) - datetime.datetime.min if isinstance(value,
                                                                                                                datetime.time) else datetime.timedelta(
            hours=value * 24)),
    document_date=lambda df: df['document_date'].astype('datetime64[ns]'),
    accounting_date=lambda df: df['accounting_date'].astype('datetime64[ns]'),
)

convert_object_cols_to_category(coating_df)
coating_df

/tmp/ipykernel_95769/3487003655.py:53: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date=lambda df: pd.to_datetime(df['date'], errors='coerce'),


'converted soc to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted cost_ci to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


...

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted line to categorical column'

'converted Ejercicio to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


soc     ce_cost production_line   cost_ci denom_cl_cst    cta_cp  \
2       TM01  30117207.0       Pintado 1  50A01353     PINTURAS  11E04012   
35      TM01  30117208.0       Pintado 2  50A01353     PINTURAS  11E04012   
36      TM01  30117208.0       Pintado 2  50A01353     PINTURAS  11E04012   
44      TM01  30117207.0       Pintado 1  50A01353     PINTURAS  11E04012   
176     TM01  30117207.0       Pintado 1  50A01353     PINTURAS  11E04012   
...      ...         ...             ...       ...          ...       ...   
218456   NaN         NaN       Pintado 2       NaN          NaN       NaN   
218457   NaN         NaN       Pintado 2       NaN          NaN       NaN   
218458   NaN         NaN       Pintado 2       NaN          NaN       NaN   
218459   NaN         NaN       Pintado 2       NaN          NaN       NaN   
218460   NaN         NaN       Pintado 2       NaN          NaN       NaN   

       account_denomination    paint_id                   paint_name  \
2         MAT PRIM-PINTURAS  U0222_AKZO        0222-AZUL RAL-5003 FC   
35        MAT PRIM-PINTURAS  I3329_AKZO  0318-TOPAZ METALLIC 14H2593   
36        MAT PRIM-PINTURAS  I3329_AKZO  0318-TOPAZ METALLIC 14H2593   
44        MAT PRIM-PINTURAS  I3052_VALS        0121-BLANCO STD KYNAR   
176       MAT PRIM-PINTURAS  I3052_VALS        0121-BLANCO STD KYNAR   
...                     ...         ...                          ...   
218456                  NaN  I0604_OSEL            0601-VERDE PRIMSA   
218457                  NaN  I1086_AKZO        0021-UNIVERSAL PRIMER   
218458                  NaN  I0177_AKZO       0104-BLANCO SUPERMATIC   
218459                  NaN  I4054_AKZO             0151-BLANCO FRIO   
218460                  NaN  I1513_VALS           0526-BLACK REF SAB   

       order_partner  ... accounting_date            time       date  \
2                NaN  ...      2020-07-24 0 days 00:12:52 2020-07-24   
35               NaN  ...      2020-07-24 0 days 22:28:26 2020-07-24   
36               NaN  ...      2020-07-24 0 days 21:28:35 2020-07-24   
44               NaN  ...      2020-07-25 0 days 14:00:07 2020-07-25   
176              NaN  ...      2020-07-25 0 days 14:32:47 2020-07-25   
...              ...  ...             ...             ...        ...   
218456           NaN  ...      2021-12-31 0 days 06:41:08 2021-12-31   
218457           NaN  ...      2021-08-12 0 days 18:40:29 2021-12-31   
218458           NaN  ...      2021-02-12 0 days 03:36:15 2021-12-02   
218459           NaN  ...      2021-02-12 0 days 03:35:01 2021-12-02   
218460           NaN  ...      2021-02-12 0 days 03:36:58 2021-12-02   

       assignment production_plant  supplier price_per_liter client  \
2        20200724              Uni      AKZO       31.580026    NaN   
35       20200724              Uni      AKZO       17.510026    NaN   
36       20200724              Uni      AKZO       17.510026    NaN   
44       20200725              Uni      VALS       15.630000    NaN   
176      20200725              Uni      VALS       15.630000    NaN   
...           ...              ...       ...             ...    ...   
218456   20211231              Uni      OSEL        3.860000    NaN   
218457   20211208              Uni      AKZO        7.480000    NaN   
218458   20211202              Uni      AKZO        6.130000    NaN   
218459   20211202              Uni      AKZO        6.209987    NaN   
218460   20211202              Uni      VALS       10.200000    NaN   

                 line  Ejercicio  
2       Pintado 1 UNI  Ej.'20/21  
35      Pintado 2 UNI  Ej.'20/21  
36      Pintado 2 UNI  Ej.'20/21  
44      Pintado 1 UNI  Ej.'20/21  
176     Pintado 1 UNI  Ej.'20/21  
...               ...        ...  
218456  Pintado 2 UNI        NaN  
218457  Pintado 2 UNI        NaN  
218458  Pintado 2 UNI        NaN  
218459  Pintado 2 UNI        NaN  
218460  Pintado 2 UNI        NaN  

[63405 rows x 37 columns]

## For paint analysis dataframe

In [30]:
# paint_analysis_df = raw_paint_analysis_df.rename(columns={
#     'Linea ': 'production_line',
#     'Mes': 'month',
#     'Mes num': 'month_number',
#     'Pintura': 'paint',
#     'Real': 'real_consumption',
#     'Teo': 'theoretical_consumption',
#     'Dif': 'consumption_difference',
#     'Rendimeinto Std': 'average_yield',
#     'Rendimeinto Real': 'real_yield',
#     'Diferencia de Rendimiento': 'yield_difference',
#     'Metros cuadrados reales': 'real_produced_square_meters',
#     'Diferencia porcentual de rendimiento': 'yield_difference_percent'
# }, errors='raise')


# convert_object_cols_to_category(paint_analysis_df)
# print_numeric_columns(paint_analysis_df)

## For paint catalog dataframe

In [31]:
paint_catalog_df = raw_paint_catalog_df.rename(columns={
    'Grupo': 'group',
    'Clave pintura': 'paint_name',  # nombre pintura
    'Familia pintura': 'paint_family',
    'Insumo': 'paint_code',
    'Proveedor': 'supplier',
    'Insumo Proveedor': ' ',
    'Producto Clase': 'product_class',
    'C COLOR PROVEE': 'probee_color_c',
    'CLAVE UNIFICADO': 'unified_key',  # codigo de la pintura
    'DESC CLEAR\n': 'clear_desc',
    'DENSIDAD': 'density',
    'PRIMER': 'primer',
    'STATUS COL PROV': 'col_prov_status',
    'LENETA': 'lenet',
    'MOLIENDA': 'grinding',
    'UNI': 'uni',
    'MVA': 'mva',
    'UNI MVA': 'uni_mva',
    'JUV': 'juv',
    'PSQ': 'psq',
    'PORC REDUCTOR': 'reductor_per',
    'RBO PINTURAS': 'rbo_paints',
    'SOLVENTE 1': 'solvent_1',  # solventes con los que es compatible la pintura
    'SOLVENTE 2': 'solvent_2',
    'SOLVENTE 3': 'solvent_3',
    'RENDIMIENTO': 'paint_catalog_yield',  # rendimiento indicado por el proveedor
    'REV RECOMENDADA': 'recommended_rev',
    'SOL PESO': 'solid_by_weight',  # solidos por peso. todo lo que no es liquido
    'SOL VOLUMEN': 'solid_by_volume',  # solidos por volumen. todo lo que no es liquido
    'SOLVENTE ADITIV': 'adi_solvent',
    'SUSTRATO 1': 'substratum_1',
    'SUSTRATO 2': 'substratum_2',
    'SUSTRATO 3': 'substratum_3',
    'TEMP METAL': 'metal_temp',
    'TEMP VISCOSIDAD': 'viscosity_temp',
    'TIEMP METAL SEG': 'metal_seg_tiemp',
    'DESC TINTA': 'ink_desc',
    'DRYPLACE': 'dryplace',
    'DRYPLACE ROHS': 'dryplace_rohs',
    'F ZINC': 'zinc_f',
    'FOSFA HIERRO': 'iron_fosfat',
    'F HIERRO ROH': 'iron_roh',
    'ZINC ROHS': 'zinc_rohs',
    'OXIDO COMP': 'oxide_comp',
    'OXIDO ROHS': 'oxide_rohs',
    'VISCOSIDAD': 'viscosity',  # se toma como rango de tiempo
    'CAPA SECA PNT MIN': 'paint_dry_coat_min',
    'CAPA SECA PNT MAX': 'paint_dry_coat_max',
    'CPASECA 1ERPASE MIN': 'first_pass_drycoat_min',
    'CPASECA 1ERPASE MAX': 'first_pass_drycoat_max',
    'BRILLO PNT MIN': 'min_paint_brightness',
    'BRILLO PNT MAX': 'max_paint_brightness',
    'BRILLO 1ERPASE MIN': 'first_pass_brightness_min',
    'BRILLO 1ERPASE MAX': 'first_pass_brightness_max',
    'ANGL BRILLO PNT': 'brightness_angle_paint',
    'ANGLBRI 1ERPASE': 'first_angle_pass',
    'CURADO PNT': 'cured_paint',
    'CURADO 1ERPASE': 'cured_first_pass',
    'DUREZA LAP PNT MIN': 'toughness_paint_min',
    'DUREZA LAP PNT MAX': 'toughness_paint_max',
    'DRZALAP 1ERPASE MIN': 'first_pass_drzalap_min',
    'DRZALAP 1ERPASE MAX': 'first_pass_drzalpa_max',
    'IMPACTO  PNT MIN': 'paint_impact_min',
    'IMPACTO PNT MAX': 'paint_impact_max',
    'IMPACTO 1ERPASE MIN': 'first_pass_impact_min',
    'IMPACTO 1ERPASE MAX': 'first_pass_impact_max',
    'FLEXIÓN NTO PNT MIN': 'flex_paint_min',
    'FLEXIÓN NTO PNT MAX': 'flex_paint_max',
    'FLEXNTO 1ERPASE MIN': 'first_pass_flex_min',
    'FLEXNTO 1ERPASE MAX': 'first_pass_flex_max',
    'FLEXIÓN NMC PNT MIN': 'paint_flex_nmc_min',
    'FLEXIÓN NMC PNT MAX': 'paint_flex_nmc_max',
    'REFLECT PNT': 'paint_reflection',
    'DELTA L PNT MIN': 'min_paint_delta_l',
    'DELTA L PNT MAX': 'max_paint_delta_l',
    'DELTA A PNT MIN': 'min_paint_delta_a',
    'DELTA A PNT MAX': 'max_paint_delta_a',
    'DELTA B PNT MIN': 'min_paint_delta_b',
    'DELTA B PNT MAX': 'max_paint_delta_b',
    'DELTA E PNT MIN': 'min_paint_delta_e',
    'DELTA E PNT MAX': 'max_paint_delta_e',
    'DELTA L 1ERPASE MIN': 'min_first_pass_delta_l',
    'DELTA L 1ERPASE MAX': 'max_first_pass_delta_l',
    'DELTA A 1ERPASE MIN': 'min_first_pass_delta_a',
    'DELTA A 1ERPASE MAX': 'max_first_pass_delta_a',
    'DELTA L LAB PNT MIN': 'min_paint_lab_delta_l',
    'DELTA L LAB PNT MAX': 'max_paint_lab_delta_l',
    'DELTA A LAB PNT MIN': 'min_paint_lab_delta_a',
    'DELTA A LAB PNT MAX': 'max_paint_lab_delta_a',
    'DELTA B 1ERPASE MIN': 'min_first_pass_delta_b',
    'DELTA B 1ERPASE MAX': 'max_first_pass_delta_b',
    'DELTA B LAB PNT MIN': 'min_paint_lab_delta_b',
    'DELTA B LAB PNT MAX': 'max_paint_lab_delta_b',
    'DELTA E 1ERPASE MIN': 'min_first_pass_delta_e',
    'DELTA E 1ERPASE MAX': 'max_first_pass_delta_e',
    'DELTA C PNT MIN': 'min_paint_delta_c',
    'DELTA C PNT MAX': 'max_paint_delta_c',
    'DELTA C 1ERPASE MIN': 'min_delta_c_first_pass',
    'DELTA C 1ERPASE MAX': 'max_delta_c_first_pass',
    'DELTA C LAB PNT MIN': 'min_paint_lab_delta_c',
    'DELTA C LAB PNT MAX': 'max_paint_lab_delta_c',
    'ILUMINANTE PNT': 'paint_iluminate',
    'ILUMNTE 1ERPASE': 'first_paint_iluminate',
    'DA LAB PP PNT MIN': 'da_lab_pp_min',
    'DA LAB PP PNT MAX': 'da_lab_pp_max',
    'DB LAB PP PNT MIN': 'db_lab_pp_min',
    'DB LAB PP PNT MAX': 'db_lab_pp_max',
    'DL LAB PP PNT MIN': 'dl_lab_pp_min',
    'DL LAB PP PNT MAX': 'dl_lab_pp_max',
    'REFLECT PNT MIN': 'min_paint_reflect',
    'REFLECT PNT MAX': 'max_paint_reflect',
    'DUREZA MEC PNT MIN': 'max_mec_paint_toughness',
    'DUREZA MEC PNT MAX': 'min_mec_paint_toughness',
    'BRILLO ORIENT': 'orient_brightness',
    'DELTA A 45° LAB': 'delta_a_45_lab',
    'DELTA A 45°': 'delta_a_45',
    'DELTA B 45° LAB': 'delta_b_45_lab',
    'DELTA B 45°': 'delta_b_45',
    'DELTA E 45°': 'delta_e_45',
    'DELTA L 45° LAB': 'delta_l_lab',
    'DELTA L 45°': 'delta_l_45',
    'EMISIVIDAD TERMICA': 'terminc_emissivity',
    'Sólidos por Volumen Mínimo [%]': 'solids_per_min_volume',
    'Capa Seca Nominal [mils]': 'nominal_dry_coat',
    'Rendimiento Shreveport [m2/L]': 'shreveport_yield',
    'Rendimiento Canning [m2/L]': 'canning_yield'
    # rendimiento que ternium asocia a estas pinturas. checar esta columna
}, errors='raise')

# Change paint catalog columns
paint_catalog_df = paint_catalog_df.assign(
    density=lambda df: df['density'].replace('1.94 - 2.42.00', '1.94 - 2.42'),
)
paint_catalog_df = paint_catalog_df.assign(
    group=lambda df: df['group'].astype('category'),
    density=lambda df: df['density'].map(lambda x: sum(map(float, x.split('-'))) if isinstance(x, str) else x),
    solid_by_weight=lambda df: df['solid_by_weight'].map(
        lambda x: sum(map(float, x.split('-'))) if isinstance(x, str) else x),
    solid_by_volume=lambda df: df['solid_by_volume'].map(
        lambda x: sum(map(float, x.split('-'))) if isinstance(x, str) else x),
    metal_temp=lambda df: df['metal_temp'].map(lambda x: sum(map(float, x.split('-'))) if isinstance(x, str) else x),
    viscosity=lambda df: df['viscosity'].map(lambda x: sum(map(float, x.split('-'))) if isinstance(x, str) else x), )

convert_object_cols_to_category(paint_catalog_df)


'converted paint_name to categorical column'

'converted paint_family to categorical column'

'converted paint_code to categorical column'

'converted supplier to categorical column'

'converted   to categorical column'

...

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted first_paint_iluminate to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


'converted orient_brightness to categorical column'

/tmp/ipykernel_95769/905205161.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df[name] = df[name].astype('str').astype('category').replace('nan', value=np.nan)


## For paint square meters dataframe

In [32]:
paint_square_meters_df = raw_paint_square_meters_df.rename(columns={
    'Linea': 'production_line',
    'Mes': 'month',
    'Num mes': 'month_number',
    'Pintura': 'paint_name',
    'Metros cuadrados reales (m2)': 'real_square_meters',
}, errors='raise')
convert_object_cols_to_category(paint_square_meters_df)

'converted production_line to categorical column'

'converted month to categorical column'

'converted paint_name to categorical column'

## Extract paint properties from paint catalog

In [33]:
paint_properties_df = paint_catalog_df._get_numeric_data().assign(paint_name=paint_catalog_df['paint_name']).drop([
    'reductor_per',
    'viscosity_temp',
], axis='columns').groupby(
    'paint_name').mean()

/tmp/ipykernel_95769/3563865373.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ], axis='columns').groupby(


# Grouping
## For production dataframe

In [61]:
# Separated production by paint type
inferior_clear_production_df = production_df.copy().drop(
    columns=['superior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code',
             'inferior_primer_code'], axis=1).rename(columns={'inferior_clear_code': 'paint_name'})
superior_clear_production_df = production_df.copy().drop(
    columns=['inferior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code',
             'inferior_primer_code'], axis=1).rename(columns={'superior_clear_code': 'paint_name'})
inferior_color_production_df = production_df.copy().drop(
    columns=['inferior_clear_code', 'superior_clear_code', 'superior_color_code', 'superior_primer_code',
             'inferior_primer_code'], axis=1).rename(columns={'inferior_color_code': 'paint_name'})
superior_color_production_df = production_df.copy().drop(
    columns=['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_primer_code',
             'inferior_primer_code'], axis=1).rename(columns={'superior_color_code': 'paint_name'})
inferior_primer_production_df = production_df.copy().drop(
    columns=['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code',
             'superior_primer_code'], axis=1).rename(columns={'inferior_primer_code': 'paint_name'})
superior_primer_production_df = production_df.copy().drop(
    columns=['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code',
             'inferior_primer_code'], axis=1).rename(columns={'superior_primer_code': 'paint_name'})
separated_production_df = pd.concat(
    [inferior_clear_production_df, superior_clear_production_df, inferior_color_production_df,
     superior_clear_production_df, inferior_primer_production_df, superior_clear_production_df], ignore_index=True)
separated_production_df.dropna(subset='paint_name', inplace=True)

separated_production_df = separated_production_df.join(paint_performance_df, on='paint_name')

# Grouped paint production by paint and date
date_column = pd.Grouper(key='start_date', freq='D')
paint_code_color = separated_production_df['paint_name']
production_line_column = separated_production_df['production_line']

paint_production_per_date_summed_df = separated_production_df[
    [
        'input_weight_kg',
        'weight_kg',
        'start_date',
        # 'forced_balance',
        # 'forced_balance_by_user', 
        # 'registered_weight_by_user',
        'length_m',
        # 'length_plc',
        'm2',
        'polyethylene_applied_meters',

    ]].groupby(
    by=[paint_code_color, production_line_column, date_column], observed=True).sum()

paint_production_per_date_averaged_df = separated_production_df[
    [
        'start_date',
        'width_mm',
        'expected_yield',
        'thickness_mm',
        'min_weight',
        'max_weight',
        'caliber',
        'finish_coat_inf',
        'finish_coat_lo_inf',
        'finish_coat_lm_inf',
        'finish_coat_sup',
        'finish_coat_lo_sup',
        'finish_coat_lm_sup',
        'primer_coat_inf',
        'primer_coat_lo_inf',
        'primer_coat_lm_inf',
        'primer_coat_sup',
        'primer_coat_lo_sup',
        'primer_coat_lm_sup',
        'clear_coat_inf',
        'clear_coat_lo_inf',
        'clear_coat_mn_inf',
        'clear_coat_sup',
        'clear_coat_lo_sup',
        'clear_coat_lm_inf',
        'gloss_sup',
        'gloss_inf',
        'gloss_grade_sup',
        'gloss_grade_inf',
        'paint_impact_inf',
        'paint_impact_lo_inf',
        'paint_impact_lm_inf',
        'paint_impact_sup',
        'paint_impact_lo_sup',
        'paint_impact_lm_sup',
        'solvent_resistance_sup',
        'solvent_resistance_inf',
        'tone_analysis_da_inf',
        'tone_analysis_da_sup',
        'tone_analysis_db_inf',
        'tone_analysis_db_sup',
        'tone_analysis_dc_inf',
        'tone_analysis_dc_sup',
        'tone_analysis_de_inf',
        'tone_analysis_de_sup',
        'tone_analysis_dl_inf',
        'tone_analysis_dl_sup',
        'weight_ratio',
        'polyethylene_width',
        'average_speed',
        'operating_line_percent',
        'primer_sup_exit_percent',
        'primer_inf_exit_percent',
        'chromate_inf_exit_percent',
        'chromate_sup_exit_percent',
        'finish_inf_exit_percent',
        'finish_sup_exit_percent',
        'heads_usage_percent',
        'average_process_speed',
        'finish_2_inf_exit_percent',
        'finish_sup_north_exit_percent',
        'finish_sup_south_exit_percent',
    ]].groupby(
    by=[paint_code_color, production_line_column, date_column], observed=True).mean()

paint_production_per_date_df = paint_production_per_date_summed_df.join(paint_production_per_date_averaged_df)
paint_production_per_date_df.rename(columns={
    'thickness_mm': 'avg_thickness_mm',
}, inplace=True)
paint_production_per_date_df.index.names = ['paint_name', 'production_line', 'month', 'date']

ValueError: Length of names must match number of levels in MultiIndex.

## For coating dataframe

In [ ]:
month_column = pd.Grouper(key='date', freq='MS')
paint_code = coating_df['paint_name']
production_line_column2 = coating_df['production_line']

coating_mean_columns = ['transaction_monetary_value', 'price_per_liter', 'date']
coating_sum_columns = [
    'object_monetary_value',
    'monetary_value_usd',
    'total_liters_used',
    'date',
]

coating_per_paint_date_summed_df = coating_df[coating_sum_columns].groupby(
    by=[paint_code, production_line_column2, date_column], observed=True).sum()
coating_per_paint_date_averaged_df = coating_df[coating_mean_columns].groupby(
    by=[paint_code, production_line_column2, date_column], observed=True).mean()

coating_per_paint_date_df = coating_per_paint_date_summed_df.join(coating_per_paint_date_averaged_df)
coating_per_paint_date_df.index.names = ['paint_name', 'production_line', 'user', 'month', 'date']

In [ ]:
paint_production_per_date_df.columns

# Generation of final dataframes
## Raw data

In [ ]:
# Joined paint production and coating, by paint and date
paint_per_date_df = paint_production_per_date_df.join(coating_per_paint_date_df, how='inner')
paint_per_date_df = paint_per_date_df.join(paint_properties_df, how='inner')

paint_per_date_df = paint_per_date_df.assign(
    real_yield=lambda df: df['m2'] / df['total_liters_used'],
    yield_difference=lambda df: df['expected_yield'] - df['real_yield'],
    abs_yield_difference=lambda df: (df['yield_difference']).abs(),
    yield_ratio=lambda df: df['real_yield'] / df['expected_yield'],
    monetary_value_difference_usd=lambda df: (
            -1 * df['monetary_value_usd'] * (1 - df['yield_ratio']))
)

column_filtering_pipeline = pipeline.Pipeline([
    ('remove_nulls', NullColumnRemover(0.05)),
])

scaler = preprocessing.StandardScaler()

cleaning_pipeline = pipeline.Pipeline([
    ('normalization', scaler),
    ('min_max_imputation', CutoffTransformer(1.96)),
    ('denormalization', preprocessing.FunctionTransformer(
        lambda x: scaler.inverse_transform(x),
        feature_names_out='one-to-one'
    )),
    ('variance threshold', feature_selection.VarianceThreshold()),
])

filtered_paint_per_date_df = column_filtering_pipeline.fit_transform(paint_per_date_df)
columns = column_filtering_pipeline.get_feature_names_out(input_features=paint_per_date_df.columns.values)
filtered_paint_per_date_df = pd.DataFrame(
    filtered_paint_per_date_df,
    columns=columns,
    index=paint_per_date_df.index)

paint_per_date_numeric_df = filtered_paint_per_date_df._get_numeric_data()
result = cleaning_pipeline.fit_transform(paint_per_date_numeric_df)
columns = cleaning_pipeline.get_feature_names_out(input_features=paint_per_date_numeric_df.columns.values)
paint_per_date_df = pd.DataFrame(result, columns=columns, index=filtered_paint_per_date_df.index).join(
    filtered_paint_per_date_df.drop(paint_per_date_numeric_df.columns.values, axis=1))

paint_per_date_df

In [62]:
paint_per_date_df = paint_per_date_df.drop(paint_per_date_df[paint_per_date_df['real_yield'] < 0].index)

## Clean data

# Write outputs as feather files

In [63]:
feather.write_feather(coating_df, 'data/coating.feather')
feather.write_feather(paint_square_meters_df, 'data/paint_square_meters.feather')
feather.write_feather(paint_performance_df, 'data/paint_performance.feather')
feather.write_feather(production_df, 'data/production.feather')
feather.write_feather(defects_df, 'data/defects.feather')
feather.write_feather(paint_per_date_df, 'data/paint_per_date.feather')
feather.write_feather(paint_catalog_df, 'data/paint_catalog.feather')

input_weight_kg  \
paint_name                     production_line month      date       user                          
0021-UNIVERSAL PRIMER          Pintado 2       2023-06-01 2023-06-28 RHOLTEN        89298.000000   
0028-CLEAR P/ESPUMA            Pintado 1       2021-02-01 2021-02-11 ALEMASTER     113231.000000   
                                               2021-09-01 2021-09-30 RHOLTEN       238470.000000   
                                               2023-03-01 2023-03-09 RHOLTEN       455246.000000   
                               Pintado 2       2021-05-01 2021-05-18 ALEMASTER     280202.000000   
                                               2021-08-01 2021-08-30 ALEMASTER     211301.000000   
                                               2021-10-01 2021-10-30 RHOLTEN       272776.000000   
                                               2023-05-01 2023-05-14 ALEINSUMOS    224996.000000   
0039-CR FREE POLYURETAN PRIMER Pintado 2       2021-12-01 2021-12-14 ALEINSUMOS    693951.000000   
                                                          2021-12-28 ALEINSUMOS    730877.000000   
                                               2022-04-01 2022-04-03 ALEINSUMOS    541878.000000   
                                               2022-05-01 2022-05-06 ALEINSUMOS    596810.000000   
0054-CLEAR KRYSTAL KOTE        Pintado 1       2021-02-01 2021-02-28 ALEMASTER     289737.000000   
                                                                     RHOLTEN       289737.000000   
                                               2022-07-01 2022-07-31 RHOLTEN       784923.000000   
                                               2022-08-01 2022-08-12 RHOLTEN       801742.821384   
                                                          2022-08-30 RHOLTEN       667224.000000   
                               Pintado 2       2022-06-01 2022-06-02 ALEINSUMOS    801742.821384   
                                                          2022-06-04 ALEINSUMOS    801742.821384   
                                                          2022-06-05 ALEINSUMOS    801742.821384   
                                                          2022-06-19 ALEINSUMOS    737852.320691   
                                                          2022-06-21 ALEINSUMOS    799131.579220   
0099-CLEAR P/ESPUMA ROHS       Pintado 2       2021-03-01 2021-03-10 RHOLTEN       197377.000000   
0400-GRIS FONDO                Pintado 2       2022-10-01 2022-10-28 RHOLTEN       801742.821384   
1026-CLEAR PRINT INOX          Pintado 1       2022-06-01 2022-06-03 ALEINSUMOS    117114.000000   

                                                                                     weight_kg  \
paint_name                     production_line month      date       user                        
0021-UNIVERSAL PRIMER          Pintado 2       2023-06-01 2023-06-28 RHOLTEN      27955.000000   
0028-CLEAR P/ESPUMA            Pintado 1       2021-02-01 2021-02-11 ALEMASTER    71170.000000   
                                               2021-09-01 2021-09-30 RHOLTEN      80734.000000   
                                               2023-03-01 2023-03-09 RHOLTEN     281311.000000   
                               Pintado 2       2021-05-01 2021-05-18 ALEMASTER   121585.000000   
                                               2021-08-01 2021-08-30 ALEMASTER   183703.000000   
                                               2021-10-01 2021-10-30 RHOLTEN      94745.000000   
                                               2023-05-01 2023-05-14 ALEINSUMOS   74240.000000   
0039-CR FREE POLYURETAN PRIMER Pintado 2       2021-12-01 2021-12-14 ALEINSUMOS  313346.000000   
                                                          2021-12-28 ALEINSUMOS  263874.000000   
                                               2022-04-01 2022-04-03 ALEINSUMOS  240102.000000   
                                               2022-05-01 2022-05-06 ALEINSUMOS  203505.000000   
0054-CLE